### Goal:
Implement FULL CRUD for one resource
using clean structure and correct REST behavior


#### Fields
- uid (UUID)            → public identifier
- name (str)            → required
- price (float)         → required, > 0
- is_active (bool)      → default True
- created_at (datetime)


In [1]:
"""
    Resource: Product

"""

'\n    Resource: Product\n\n'

In [2]:
"""
    1. Databse Model
"""

import uuid
from datetime import datetime
from sqlalchemy.ext.asyncio import AsyncSession, async_sessionmaker
from sqlmodel import SQLModel, Field

class Product(SQLModel, table=True):
    __tablename__="product"
    uid: uuid.UUID | None = Field(default_factory=uuid.uuid4, primary_key=True, index=True)
    name: str = Field(unique=True, nullable=False, index=True)
    price: float = Field(nullable=False)
    is_active: bool = Field(default=True, nullable=False, index=True)
    created_at: datetime = Field(default_factory=datetime.now, nullable=False)
    
    
async def get_session() -> AsyncSession:
    async with async_sessionmaker() as session:
        yield session

In [3]:
"""
    2. Schema
"""
from datetime import datetime
from typing import Optional
from pydantic import BaseModel

'''Output Schema'''
class ProductRead(BaseModel):
    uid: uuid.UUID
    name: str
    price: float
    is_active: bool
    created_at: datetime
    
'''Input Schema'''
class ProductCreate(BaseModel):
    name: str
    price: float


class ProductPutUpdate(BaseModel):
    name: str
    price: float
    is_active: bool

class ProductPatchUpdate(BaseModel):
    name: Optional[str] = None
    price: Optional[float] = None
    is_active: Optional[bool] = None

In [ ]:
""" 
    3. Services
"""
import uuid
from sqlmodel import select
from sqlalchemy.ext.asyncio import AsyncSession

class ProductService:
    async def read_products(self, session: AsyncSession, limit: int = 10, offset: int = 0):
        statement = select(Product).limit(limit).offset(offset)
        result = await session.execute(statement)
        products = result.scalars().all()

        return products
    
    
    async def search_by_id(self, product_uid: uuid.UUID, session: AsyncSession):
        statement = select(Product).where(Product.uid == product_uid)
        result = await session.execute(statement)
        product = result.scalar_one_or_none()
        
        if not product:
            raise ValueError("Product not found")
        
        return product
    
    
    async def create_product(self, payload: ProductCreate, session: AsyncSession):
        statement = select(Product).where(Product.name == payload.name)
        result = await session.execute(statement)
        product = result.scalar_one_or_none()
        
        if product:
            raise ValueError("Product already exists!")
        
        new_product = Product(name=payload.name,
                              price=payload.price)
        
        session.add(new_product)
        await session.commit()
        await session.refresh(new_product)
        
        return new_product
    
    
    async def update_put_product(self, product_uid: uuid.UUID, payload: ProductPutUpdate, session: AsyncSession):
        product = await self.search_by_id(product_uid, session)
        
        if not product:
            raise ValueError("Product not found")
        
        product.name = payload.name
        product.price = payload.price
        product.is_active = payload.is_active
        
        session.add(product)
        await session.commit()
        await session.refresh(product)
        
        return product
    
    
    async def update_patch_product(self, product_uid: uuid.UUID, payload: ProductPatchUpdate, session: AsyncSession):
        product = await self.search_by_id(product_uid, session)
        
        if not product:
            raise ValueError("Product not found")
        
        data = payload.model_dump(exclude_unset=True)
        if not data:
            raise ValueError("No field provided to update")
        
        for field, value in data.items():
            setattr(product, field, value)
            
            
        session.add(product)
        await session.commit()
        await session.refresh(product)
        
        return product
        
    
    async def delete_product(self, product_uid: uuid.UUID, session:AsyncSession):
        product = await self.search_by_id(product_uid, session)
        
        if not product:
            raise ValueError("Product not found")
        
        if not product.is_active:
            return product
        
        product.is_active = False
        session.add(product)
        await session.commit()
        await session.refresh(product)

        return product

In [ ]:
""" 
    4. Routes
"""

from fastapi import FastAPI, HTTPException, Query, status, Depends
from sqlalchemy.ext.asyncio import AsyncSession


app = FastAPI()
product_service = ProductService()

@app.get("/products/", response_model=list[ProductRead], status_code=status.HTTP_200_OK)
async def fetch_all_product(session: AsyncSession = Depends(get_session),
                            limit: int = Query(10, ge=1, le=100),
                            offset: int = Query(0, ge=0)):
    try:
        products = await product_service.read_products(session)    
        return products
    
    except Exception as e:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Products not found")


@app.get("/products/{product_id}", response_model=ProductRead, status_code=status.HTTP_200_OK)
async def fetch_product(product_id: uuid.UUID, session: AsyncSession = Depends(get_session)):
    try:
        product = await product_service.search_by_id(product_id, session)
        return product
    
    except Exception as e:
        raise HTTPException(status_code=404, detail=str(e))


@app.post("/products/create", response_model=ProductRead, status_code=status.HTTP_201_CREATED)
async def create_product(payload: ProductCreate, session: AsyncSession = Depends(get_session)):
    try:
        product = await product_service.create_product(payload, session)
        return product
    
    except Exception as e:
        raise HTTPException(status_code=404, detail=str(e))



@app.put("/products/{product_id}", response_model=ProductRead, status_code=status.HTTP_200_OK)
async def update_put(product_id: uuid.UUID, payload: ProductPutUpdate, session: AsyncSession = Depends(get_session)):
    try:
        product = await product_service.update_put_product(product_id, payload, session)
        return product
    
    except Exception as e:
        raise HTTPException(status_code=404, detail=str(e))



@app.patch("/products/{product_id}", response_model=ProductRead, status_code=status.HTTP_200_OK)
async def update_patch(product_id: uuid.UUID, payload: ProductPatchUpdate, session: AsyncSession = Depends(get_session)):
    try:
        product = await product_service.update_patch_product(product_id, payload, session)
        return product
    
    except Exception as e:
        raise HTTPException(status_code=404, detail=str(e))

    

@app.delete("/products/{product_id}", status_code=status.HTTP_200_OK)
async def delete_product(product_id: uuid.UUID, session: AsyncSession = Depends(get_session)):
    try:
        product = await product_service.delete_product(product_id, session)
        return product
    
    except Exception as e:
        raise HTTPException(status_code=404, detail=str(e))


In [ ]:
"""
    5. Runing Fastapi file
"""
# Source - https://stackoverflow.com/a
# Posted by Chris, modified by community. See post 'Timeline' for change history
# Retrieved 2026-01-03, License - CC BY-SA 4.0

import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app, host="127.0.0.1", port=8001)
    server = uvicorn.Server(config)
    await server.serve()


INFO:     Started server process [17652]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:61368 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:61368 - "GET /openapi.json HTTP/1.1" 200 OK
